### postprocess arise 

Alistair Duffey, November 2024

CESM2-WACCM

N.B.: this runs over an unfinished archive of the CESM2 data moved onto CEDA

In [29]:
import os
import glob
import pandas as pd
import numpy as np
import xarray as xr
from xmip.preprocessing import rename_cmip6
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm

### options

# Model
model = 'CESM2-WACCM'

# time-periods over which to take means
assessment_periods = {'SAI':slice('2050', '2069'),
                      'Background_warming':slice('2050', '2069'),
                      'Baseline':slice('2014', '2033'), ### NEED TO UPDATE FOR CESM
                      'Pre-industrial':slice('1850', '1900')}

## note that CESM ARISE data is not CMOPR-ized
## so we need to translate variable names 
## e.g. TREFHT > tas
cesm_cmor_variable_names = {'TREFHT':'tas'}

cesm_vars = ['TREFHT'] ## for now, we just one var as only have a small archive on ceda

ensemble_members_CESM = ['001', '002', '003', '004', '005',
                         '006', '007', '008', '009', '010']

experiment_paths_CESM = {'ARISE':'/gws/nopw/j04/cpom/aduffey/ARISE/CESM/arise/',
                         'SSP245':'/gws/nopw/j04/cpom/aduffey/ARISE/CESM/ssp245/'}

# seasons
seasons = ['DJF', 'MAM', 'JJA', 'SON']

In [30]:
### get data

def get_ssp245_or_arise_ds(variable, scenario, ens_mems):
    """ returns dataset with 10 members, each running over ssp245 2015-2100 """
    
    ds_list = []
    for es in ens_mems:
        path = experiment_paths_CESM[scenario] + '/{v}/'.format(v=variable)
        ds = rename_cmip6(xr.open_mfdataset(path+'*.nc'))
        if 'height' in ds.variables:
            ds = ds.drop_vars('height')
        if 'type' in ds.variables:
            ds = ds.drop_vars('type')
            
        ## rename 
        ds_list.append(ds)
    
    DS = xr.concat(ds_list, dim='Ensemble_member')
    return DS


In [26]:
get_ssp245_or_arise_ds('TREFHT', 'ARISE', ensemble_members_CESM)

<xarray.Dataset> Size: 946MB
Dimensions:       (Ensemble_member: 10, time: 420, zlon: 1, nbnd: 2, y: 192,
                   lev: 70, ilev: 71, x: 288)
Coordinates:
  * zlon          (zlon) float64 8B 0.0
  * time          (time) object 3kB 2035-02-01 00:00:00 ... 2070-01-01 00:00:00
  * y             (y) float64 2kB -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lev           (lev) float64 560B 5.96e-06 9.827e-06 1.62e-05 ... 976.3 992.6
  * ilev          (ilev) float64 568B 4.5e-06 7.42e-06 1.223e-05 ... 985.1 1e+03
  * x             (x) float64 2kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
Dimensions without coordinates: Ensemble_member, nbnd
Data variables: (12/27)
    zlon_bnds     (Ensemble_member, time, zlon, nbnd) float64 67kB dask.array<chunksize=(1, 420, 1, 2), meta=np.ndarray>
    gw            (Ensemble_member, time, y) float64 6MB dask.array<chunksize=(1, 420, 192), meta=np.ndarray>
    hyam          (Ensemble_member, time, lev) float64 2MB dask.array<chunksize=(1, 420, 70), meta=np.ndarray>
    hybm          (Ensemble_member, time, lev) float64 2MB dask.array<chunksize=(1, 420, 70), meta=np.ndarray>
    P0            (Ensemble_member, time) float64 34kB 1e+05 1e+05 ... 1e+05
    hyai          (Ensemble_member, time, ilev) float64 2MB dask.array<chunksize=(1, 420, 71), meta=np.ndarray>
    ...            ...
    n2ovmr        (Ensemble_member, time) float64 34kB dask.array<chunksize=(1, 420), meta=np.ndarray>
    f11vmr        (Ensemble_member, time) float64 34kB dask.array<chunksize=(1, 420), meta=np.ndarray>
    f12vmr        (Ensemble_member, time) float64 34kB dask.array<chunksize=(1, 420), meta=np.ndarray>
    sol_tsi       (Ensemble_member, time) float64 34kB dask.array<chunksize=(1, 420), meta=np.ndarray>
    nsteph        (Ensemble_member, time) int32 17kB dask.array<chunksize=(1, 420), meta=np.ndarray>
    TREFHT        (Ensemble_member, time, y, x) float32 929MB dask.array<chunksize=(1, 1, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.010
    logname:           geostrat
    host:              cheyenne6
    initial_file:      b.e21.BWSSP245cmip6.f09_g17.CMIP6-SSP2-4.5-WACCM.005.c...
    topography_file:   /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/fv_0.9x1...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  month_1

In [31]:
def get_time_period(ds, slice_label):
    ds_out = ds.sel(time=assessment_periods[slice_label])
    ds_out.attrs['t_bnds'] = str(assessment_periods[slice_label].start+'_'+assessment_periods[slice_label].stop)
    return ds_out

In [32]:
def process_and_save_maps(ds, ds_seasonal, var, table, label, seasons=seasons):
    """ 
    Inputs
    ds: a time resolved, quarterly resampled, spatial dataset, with an ensemble_member dimension
    label: 'SSP245_baseline', 'ARISE', or 'SSP245_background'. Defines naming of outputs. 
    
    Function saves the mean and standard deviation across the whole time+ens_mems combined dimension
    """
    path = 'pp_archive/ARISE/CESM2-WACCM/maps/{l}/{t}/{v}/'.format(l=label, t=table, v=var)
    os.makedirs(path+'/std/', exist_ok=True)
    os.makedirs(path+'/mean/', exist_ok=True)
    
    t_bnds = ds_seasonal.t_bnds
    for season in seasons:
        ds_season = ds_seasonal.where(ds_seasonal.time.dt.season == season, drop=True)
        std = ds_season.std(dim=['time', 'Ensemble_member'])
        mean = ds_season.mean(dim=['time', 'Ensemble_member'])
        
        std.to_netcdf(path + '/std/' +'{v}_{l}_{s}_{t}_std.nc'.format(v=var, l=label, s=season, t=t_bnds))
        mean.to_netcdf(path + '/mean/' + '{v}_{l}_{s}_{t}_mean.nc'.format(v=var, l=label, s=season, t=t_bnds))

    t_bnds = ds.t_bnds
    # repeat for the annual mean:
    std = ds.std(dim=['time', 'Ensemble_member'])
    mean = ds.mean(dim=['time', 'Ensemble_member'])
    
    std.to_netcdf(path + '/std/' + '{v}_{l}_annual_{t}_std.nc'.format(v=var, l=label, t=t_bnds))
    mean.to_netcdf(path + '/mean/' + '{v}_{l}_annual_{t}_mean.nc'.format(v=var, l=label, t=t_bnds))
    return

In [35]:
# MAIN

for var in tqdm(cesm_vars):

    print(var)
    # get data
    ds_ssp = get_ssp245_or_arise_ds(var, 'SSP245', ensemble_members_CESM)
    ds_arise = get_ssp245_or_arise_ds(var, 'ARISE', ensemble_members_CESM)
    
    ds_ssp_seasonal = ds_ssp.resample(time="QS-DEC").mean()
    ds_arise_seasonal = ds_arise.resample(time="QS-DEC").mean()

    # process into time slice means:
    ssp_baseline, ssp_baseline_seasonal = get_time_period(ds_ssp, 'Baseline'), get_time_period(ds_ssp_seasonal, 'Baseline')
    ssp_background, ssp_background_seasonal = get_time_period(ds_ssp, 'Background_warming'), get_time_period(ds_ssp_seasonal, 'Background_warming')
    arise_assmt, arise_assmt_seasonal = get_time_period(ds_arise, 'SAI'), get_time_period(ds_arise_seasonal, 'SAI')

    process_and_save_maps(ssp_baseline, ssp_baseline_seasonal, 
                          var=var, table='Amon', 
                          label='SSP245_baseline', seasons=seasons)
    process_and_save_maps(ssp_background, ssp_background_seasonal, 
                          var=var, table='Amon', 
                          label='SSP245_background', seasons=seasons)
    process_and_save_maps(arise_assmt, arise_assmt_seasonal, 
                          var=var, table='Amon', 
                          label='ARISE_assmt', seasons=seasons)


  0%|          | 0/1 [00:00<?, ?it/s]

TREFHT


100%|██████████| 1/1 [04:57<00:00, 297.32s/it]
